In [1]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-exeu216r
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-exeu216r
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for ftfy from https://files.pythonhosted.org/packages/f4/f0/21efef51304172736b823689aaf82f33dbc64f54e9b046b75f5212d5cee7/ftfy-6.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for regex from https://files.pythonhosted.org/packages/a3/ec/6f1d7160c9dbbc4ffe87167f28ddfab359e1043acafce5407c8b11c5f581/regex-2024.4.16-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 10.7 MB/s eta 0:00:00
  Obtaining dependency information for wcwidth<0.3.0,>=0.2.12 from https://files.pythonhosted.org/packages/fd/84/fd2ba7aafacbad3c4201d395674fc6348826569d

In [4]:
from dataloader_places import PlacesDataset
from torch.utils.data import DataLoader
import clip
import torch
from tqdm import tqdm
import os
idx_to_classname ={0: 'bathroom', 1: 'bedroom', 2: 'childs_room',
                   3: 'classroom', 4: 'dressing_room', 5: 'living_room',
                   6: 'studio', 7: "swimming_pool"}

data_path = "../adversarial-sets/data/Places8_paths_and_labels_complete_train.npy"
out_path = "./outputs/"
outpath = os.path.join(out_path, ????)
os.makedirs(outpath, mode=777, exist_ok=True)
    
places_ds = [PlacesDataset(data_path,
                           onlylabels=[k]) for k in range(8)]
batch_size = 64
train_dataloaders_class = {k: DataLoader(places_ds[k],
                                         batch_size=batch_size,
                                         shuffle=False,
                                         num_workers=8) for k in range(8)}
for k in range(8):
    print(f"\nDataloader: {batch_size} batch size | {len(train_dataloaders_class[k])} batches | {len(train_dataloaders_class[k].dataset)} images")

print(clip.available_models())
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

Found 51655 images from 1 classes.
    Class 'bathroom' (0): 51655 images.
Found 100012 images from 1 classes.
    Class 'bedroom' (1): 100012 images.
Found 41849 images from 1 classes.
    Class 'childs_room' (2): 41849 images.
Found 33763 images from 1 classes.
    Class 'classroom' (3): 33763 images.
Found 21889 images from 1 classes.
    Class 'dressing_room' (4): 21889 images.
Found 89458 images from 1 classes.
    Class 'living_room' (5): 89458 images.
Found 12633 images from 1 classes.
    Class 'studio' (6): 12633 images.
Found 13547 images from 1 classes.
    Class 'swimming_pool' (7): 13547 images.

Dataloader: 64 batch size | 808 batches | 51655 images

Dataloader: 64 batch size | 1563 batches | 100012 images

Dataloader: 64 batch size | 654 batches | 41849 images

Dataloader: 64 batch size | 528 batches | 33763 images

Dataloader: 64 batch size | 343 batches | 21889 images

Dataloader: 64 batch size | 1398 batches | 89458 images

Dataloader: 64 batch size | 198 batches | 12

In [5]:
model, preprocess = clip.load("ViT-B/32", device=device)
print(model)

 38%|██████████████▊                        | 128M/338M [00:24<00:40, 5.48MiB/s]


KeyboardInterrupt: 

In [ ]:
print(preprocess)

In [42]:
for i in range(8):
    image_features = []
    text_features = []
    for inputs, labels, _ in tqdm(train_dataloaders_class[i]):
        inputs = inputs.to(device)
        text_inputs = torch.cat([clip.tokenize(
            f"a photo of a {idx_to_classname[label.item()]}", truncate=True) for label in labels]).to(device)

        # Calculate features
        with torch.no_grad():
            img_feat_batch = model.encode_image(inputs).cpu().data.numpy()
            text_feat_batch = model.encode_text(text_inputs).cpu().data.numpy()
            for idx, _ in enumerate(img_feat_batch):
                image_features.append(img_feat_batch[idx].tolist())
                text_features.append(text_feat_batch[idx].tolist())
    
    image_features = np.asarray(image_features)
    text_features = np.asarray(text_features)
    np.save(f"{outpath}/places8_image_features_clip_class_{i}.npy", image_features)
    np.save(f"{outpath}/places8_text_features_clip_class_{i}.npy", text_features)


100%|████████████████████████████████████████████████████████████████| 5701/5701 [43:10<00:00,  2.20it/s]


In [45]:
print(len(image_features))

364806

In [46]:
import numpy as np

image_features = np.asarray(image_features)
text_features = np.asarray(text_features)
np.save("places8_image_features_clip.npy", image_features)
np.save("places8_text_features_clip.npy", text_features)